# Analysis of Revised `RWC_alam()`

*Last updated by Arthur Ryman on 2022-05-19*

## Introduction

I've refactored the Python implementation of `RWC_alam()` to eliminate repeated code.
The code accepts an optional `v` parameter which is not explicitly described in WR2015.
The goal of this notebook is to create a mathematical specifcation that includes `v`
and define how to compute the initial approximation required by `nsolve()`.

In [1]:
import math
from sympy import *
from acmpy.papers.wr2015 import *

## `vshftf(v)`

```
def vshftf(v: nonnegint) -> int:
    return (2 * v + 3) ** 2
```

In [2]:
v = Symbol('v', integer=True, nonnegative=True)
v

v

In [3]:
def vshftf(v):
    return (2 * v + 3) ** 2

vshftf(v)

(2*v + 3)**2

In [4]:
vshftf(0)

9

## `muf(A, c1, c2, v)`

```
def muf(A: Expr, c1: float, c2: float, v: nonnegint) -> Expr:
    vshft: int = vshftf(v)
    return sqrt(vshft + (A * c1 / c2) ** 2)
```

In [5]:
def muf(A, c1, c2, v):
    vshft = vshftf(v)
    return sqrt(vshft + (A * c1 / c2) ** 2)

A = Symbol('A', real=True, positive=True)
muf(A, c1, c2, v)

sqrt(A**2*c1**2/c2**2 + (2*v + 3)**2)

In [6]:
muf(A, c1, c2, 0)

sqrt(A**2*c1**2/c2**2 + 9)

## `RWC1(A, B, c1, c2, v)`

```
def RWC1(A: Expr, B: float, c1: float, c2: float, v: nonnegint = 0) -> Expr:
    return A ** 3 - B ** 2 * c1 * A - (2 * v + 7) * B ** 2 * c2
```

In [7]:
def RWC1(A, B, c1, c2, v):
    return A ** 3 - B ** 2 * c1 * A - (2 * v + 7) * B ** 2 * c2

RWC1(A, B, c1, c2, v)

A**3 - A*B**2*c1 - B**2*c2*(2*v + 7)

In [8]:
RWC1(A, B, c1, c2, 0)

A**3 - A*B**2*c1 - 7*B**2*c2

## `RWC_alam_clam(B, c1, c2, v)`

```
def RWC_alam_clam(B: float, c1: float, c2: float, v: nonnegint = 0
                  ) -> tuple[float, float]:

    A: Symbol = Symbol('A', real=True)
    F1: Expr = RWC1(A, B, c1, c2, v)
    aa0: float = float(nsolve(F1, A, 20))

    return math.sqrt(aa0), 2.5
```

In this case $\lambda_v$ is independent of $A$ and takes the constant value $2.5$.
The function finds the zero of $F_1$.

In [9]:
F1 = RWC1(A, B, c1, c2, v)
F1

A**3 - A*B**2*c1 - B**2*c2*(2*v + 7)

We have the general constraint $c_2 \ge 0$.
There are two cases:
1. $c_2 = 0, c_1 > 0$
2. $c_2 > 0$

### Case 1: $c_2 = 0, c_1 > 0$

In [10]:
c1_case1 = Symbol('c1', real=True, positive=True)
c2_case1 = 0
F1_case1 = RWC1(A, B, c1_case1, c2_case1, v)
F1_case1

A**3 - A*B**2*c1

We can solve this exactly.
Recall that $A > 0$.

In [11]:
solve_case1 = solve(F1_case1, A)
solve_case1

[B*sqrt(c1)]

In [12]:
A0_solve_case1 = solve_case1[0]
A0_solve_case1

B*sqrt(c1)

In [13]:
solveset_case1 = solveset(F1_case1, A, domain=Reals)
solveset_case1

{0, -B*sqrt(c1), B*sqrt(c1)}

In [14]:
[s for s in solveset_case1 if s > 0]

[B*sqrt(c1)]

### Tests for Case 1

In [15]:
from acmpy.hamiltonian_data import A0_case1

def print_tests_case1():
    for B in [10, 20, 30]:
        for c1 in [1, 2, 3]:
            A0 = A0_case1(B, c1)
            print(f'({B}, {c1}, {A0}),')

print_tests_case1()

(10, 1, 10.0),
(10, 2, 14.142135623730951),
(10, 3, 17.32050807568877),
(20, 1, 20.0),
(20, 2, 28.284271247461902),
(20, 3, 34.64101615137754),
(30, 1, 30.0),
(30, 2, 42.42640687119285),
(30, 3, 51.96152422706631),


### Case 2: $c_2 > 0$

In [16]:
c1_case2 = c1
c2_case2 = Symbol('c2', real=True, positive=True)
F1_case2 = RWC1(A, B, c1_case2, c2_case2, v)
F1_case2

A**3 - A*B**2*c1 - B**2*c2*(2*v + 7)

For small $A$, $F_1(A)$ becomes a negative constant.
For large $A$, $F_1(A)$ is approximately $A^3$ which is positive.
It therefore has at least one positive zero.
Approximate it by dropping the $c_1$ term.

In [17]:
F1_case2_approx = F1_case2.subs(c1, 0)
F1_case2_approx

A**3 - B**2*c2*(2*v + 7)

In [18]:
solve_case2_approx = solve(F1_case2_approx, A)
solve_case2_approx

[(2*B**2*c2*v + 7*B**2*c2)**(1/3)]

In [19]:
A0_case2_approx = solve_case2_approx[0]
A0_case2_approx

(2*B**2*c2*v + 7*B**2*c2)**(1/3)

In [20]:
solve_case2 = solve(F1_case2, A)
len(solve_case2)

3

In [21]:
solve_case2[0]

B**2*c1/(3*(B**2*c2*v + 7*B**2*c2/2 + sqrt(-B**6*c1**3/27 + (-2*B**2*c2*v - 7*B**2*c2)**2/4))**(1/3)) + (B**2*c2*v + 7*B**2*c2/2 + sqrt(-B**6*c1**3/27 + (-2*B**2*c2*v - 7*B**2*c2)**2/4))**(1/3)

In [22]:
solve_case2[1]

B**2*c1/(3*(-1/2 - sqrt(3)*I/2)*(B**2*c2*v + 7*B**2*c2/2 + sqrt(-B**6*c1**3/27 + (-2*B**2*c2*v - 7*B**2*c2)**2/4))**(1/3)) + (-1/2 - sqrt(3)*I/2)*(B**2*c2*v + 7*B**2*c2/2 + sqrt(-B**6*c1**3/27 + (-2*B**2*c2*v - 7*B**2*c2)**2/4))**(1/3)

In [23]:
solve_case2[2]

B**2*c1/(3*(-1/2 + sqrt(3)*I/2)*(B**2*c2*v + 7*B**2*c2/2 + sqrt(-B**6*c1**3/27 + (-2*B**2*c2*v - 7*B**2*c2)**2/4))**(1/3)) + (-1/2 + sqrt(3)*I/2)*(B**2*c2*v + 7*B**2*c2/2 + sqrt(-B**6*c1**3/27 + (-2*B**2*c2*v - 7*B**2*c2)**2/4))**(1/3)

### Tests for Case 2

In [24]:
from acmpy.hamiltonian_data import A0_case2_approx

def print_tests_case2():
    for B in [10, 20, 30]:
        for c2 in [1, 2, 3]:
            for v in [0, 1, 2]:
                A0 = A0_case2_approx(B, c2, v)
                print(f'({B}, {c2}, {v}, {A0}),')

print_tests_case2()

(10, 1, 0, 8.879040017426005),
(10, 1, 1, 9.654893846056297),
(10, 1, 2, 10.32280115456367),
(10, 2, 0, 11.186889420813966),
(10, 2, 1, 12.164403991146798),
(10, 2, 2, 13.005914468513868),
(10, 3, 0, 12.80579164987494),
(10, 3, 1, 13.924766500838334),
(10, 3, 2, 14.888055529538272),
(20, 1, 0, 14.094597464129782),
(20, 1, 1, 15.32618864787106),
(20, 1, 2, 16.386425412012915),
(20, 2, 0, 17.75808003485201),
(20, 2, 1, 19.309787692112593),
(20, 2, 2, 20.64560230912734),
(20, 3, 0, 20.327927136297067),
(20, 3, 1, 22.104188991842317),
(20, 3, 2, 23.63331500935002),
(30, 1, 0, 18.469147504478332),
(30, 1, 1, 20.08298850246508),
(30, 1, 2, 21.472291690189408),
(30, 2, 0, 23.269667714505616),
(30, 2, 1, 25.30297995905247),
(30, 2, 2, 27.0533922899524),
(30, 3, 0, 26.637120052278018),
(30, 3, 1, 28.964681538168886),
(30, 3, 2, 30.968403463691008),


## `RWC2(A, mu, B, c1, c2, v)`

```
def RWC2(A: Expr, mu: Expr, B: float, c1: float, c2: float, v: nonnegint = 0) -> Expr:
    vshft: int = vshftf(v)
    return (c1 / c2) ** 2 * (-vshft * A ** 5 / mu ** 2
                             + A ** 3 * B ** 2 * c1
                             + A ** 2 * B ** 2 * c2 * (mu + 3)) \
           + A ** 3 * (2 * mu + vshft) \
           - B ** 2 * mu * (mu + 2) * (A * c1 + c2 * (mu + 4))
```

In [25]:
mu = Symbol('mu', real=True, positive=True)
mu

mu

In [26]:
def RWC2(A, mu, B, c1, c2, v):
    vshft = vshftf(v)
    return (c1 / c2) ** 2 * (-vshft * A ** 5 / mu ** 2
                             + A ** 3 * B ** 2 * c1
                             + A ** 2 * B ** 2 * c2 * (mu + 3)) \
           + A ** 3 * (2 * mu + vshft) \
           - B ** 2 * mu * (mu + 2) * (A * c1 + c2 * (mu + 4))


RWC2(A, mu, B, c1, c2, v)

A**3*(2*mu + (2*v + 3)**2) - B**2*mu*(mu + 2)*(A*c1 + c2*(mu + 4)) + c1**2*(-A**5*(2*v + 3)**2/mu**2 + A**3*B**2*c1 + A**2*B**2*c2*(mu + 3))/c2**2

## `RWC_alam(B, c1, c2, v)`

```
def RWC_alam(B: float, c1: float, c2: float, v: nonnegint = 0
             ) -> tuple[float, float]:
    require_nonnegint('v', v)

    if c1 >= 0:
        return RWC_alam_clam(B, c1, c2, v)

    assert c1 < 0
    A: Symbol = Symbol('A', real=True)
    mu: Expr = muf(A, c1, c2, v)
    F2: Expr = RWC2(A, mu, B, c1, c2, v)

    aa_initial: float = (35 * B ** 2 * c2 / 2) ** (1 / 3)
    aa0: float = float(nsolve(F2, A, aa_initial))

    return math.sqrt(aa0), float(1 + muf(S(aa0), c1, c2, v) / 2)
```

In this case we only need to consider $c_1 < 0, c_2 > 0$.

### Case 3: $c_1 < 0, c_2 > 0$

In [27]:
c1_case3 = Symbol('c1', real=True, negative=True)
c2_case3 = Symbol('c2', real=True, positive=True)
mu_case3 = muf(A, c1_case3, c2_case3, v)
mu_case3

sqrt(A**2*c1**2/c2**2 + (2*v + 3)**2)

In [28]:
F2_case3 = RWC2(A, mu_case3, B, c1_case3, c2_case3, v)
F2_case3

A**3*((2*v + 3)**2 + 2*sqrt(A**2*c1**2/c2**2 + (2*v + 3)**2)) - B**2*(A*c1 + c2*(sqrt(A**2*c1**2/c2**2 + (2*v + 3)**2) + 4))*sqrt(A**2*c1**2/c2**2 + (2*v + 3)**2)*(sqrt(A**2*c1**2/c2**2 + (2*v + 3)**2) + 2) + c1**2*(-A**5*(2*v + 3)**2/(A**2*c1**2/c2**2 + (2*v + 3)**2) + A**3*B**2*c1 + A**2*B**2*c2*(sqrt(A**2*c1**2/c2**2 + (2*v + 3)**2) + 3))/c2**2

In [29]:
O(F2_case3, (A, 0))

O(1)

In [30]:
F2_case3_approx_0 = F2_case3.subs(A, 0)
F2_case3_approx_0

-B**2*c2*(2*v + 3)*(2*v + 5)*(2*v + 7)

In [31]:
O(F2_case3, (A, oo))

O(A**4, (A, oo))

In [32]:
F2_case3_approx_oo = limit(F2_case3 / A ** 4, A, oo) * A ** 4
F2_case3_approx_oo

-2*A**4*c1/c2

In [33]:
F2_case3_approx = F2_case3_approx_0 + F2_case3_approx_oo
F2_case3_approx

-2*A**4*c1/c2 - B**2*c2*(2*v + 3)*(2*v + 5)*(2*v + 7)

In [34]:
solve_case3_approx = solve(F2_case3_approx, A)
solve_case3_approx

[2**(3/4)*(-1/c1)**(1/4)*(8*B**2*c2**2*v**3 + 60*B**2*c2**2*v**2 + 142*B**2*c2**2*v + 105*B**2*c2**2)**(1/4)/2]

In [35]:
A0_case3_approx = solve_case3_approx[0]
A0_case3_approx

2**(3/4)*(-1/c1)**(1/4)*(8*B**2*c2**2*v**3 + 60*B**2*c2**2*v**2 + 142*B**2*c2**2*v + 105*B**2*c2**2)**(1/4)/2

In [36]:
simplify(A0_case3_approx)

2**(3/4)*sqrt(B)*sqrt(c2)*(8*v**3 + 60*v**2 + 142*v + 105)**(1/4)/(2*(-c1)**(1/4))

In [37]:
A0_case3_approx > 0

True

### Tests for Case 3

In [45]:
A0_case3_approx_B = A0_case3_approx.subs([(c1_case3, -3.0), (c2_case3, 2.0), (v, 0)])
A0_case3_approx_B.evalf()

2.89250760851908*B**0.5

In [48]:
def print_tests_case3():
    for B_value in range(10, 101, 10):
        A0 = A0_case3_approx_B.subs(B, B_value)
        print(f'({B_value}, {float(A0)}),')

print_tests_case3()

(10, 9.146912192286944),
(20, 12.935687276168014),
(30, 15.84291664941221),
(40, 18.293824384573888),
(50, 20.453117446175234),
(60, 22.40526759314526),
(70, 24.20045492493587),
(80, 25.87137455233603),
(90, 27.440736576860832),
(100, 28.925076085190778),


In [50]:
def A0_case3_approx_fn(B, c1, c2, v):
    return math.sqrt(B * c2) * ((2 * v + 3) * (2 * v + 5) * (2 * v + 7) / (-2 * c1)) ** ( 1 / 4)

[(B, A0_case3_approx_fn(B, -3, 2, 0)) for B in range(10, 101, 10)]

[(10, 9.146912192286944),
 (20, 12.935687276168016),
 (30, 15.842916649412212),
 (40, 18.293824384573888),
 (50, 20.453117446175234),
 (60, 22.405267593145265),
 (70, 24.200454924935872),
 (80, 25.871374552336032),
 (90, 27.440736576860836),
 (100, 28.92507608519078)]